In [1]:
import datetime
import time
import json

# import psycopg2

# url = "hdfs://127.0.0.1:9000/socmed_input/"
# out_url = "hdfs://127.0.0.1:9000/socmed_output/"

# conn = psycopg2.connect(database="bigdatatest",
#     host="localhost",
#     user="geraldakbar",
#     password='bigdata'
# )

In [2]:
def normalizeTime(time):
    res = ''
    for i in range(len(time)):
        if i == 6 or i == 7:
            res += '0'
        elif i == 4:
            if int(time[i]) < 5:
                res += '0'
            else:
                res += '5'
        else:
            res += time[i] 
    return res

def parseTime(type, time):
    if type == "twitter":
        ts = t.strftime('%Y-%m-%d %H:%M:%S', t.strptime(time,'%a %b %d %H:%M:%S +0000 %Y'))
        return ts[:10]
    elif type == "instagram" or type == "other":
        time = str(datetime.datetime.fromtimestamp(int(time)).date())
        return time[:10]
    else:
        return time[:10] + ' ' + normalizeTime(time[11:19])

In [3]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

ssc = StreamingContext(sc,1) #stream each one second
ssc.checkpoint("./checkpoint")
lines = KafkaUtils.createDirectStream(ssc, ['youtube'],
                                      {"metadata.broker.list": "localhost:9092"})

def getYT(lines):
    # map the lines to a key-value pair
    def map(line):
        snippet = line['snippet']
        date = snippet['publishedAt'][0:10] + ' ' + normalizeTime(snippet['publishedAt'][11:19])
        return (("youtube", date), 1)
    
    def userMap(line):
        snippet = line['snippet']
        user = None
        # check if comment is a reply
        if('channelTitle' in snippet):
            user = snippet['channelId']
        else:
            user = snippet['authorChannelId']['value']
        date = snippet['publishedAt'][0:10] + ' ' + normalizeTime(snippet['publishedAt'][11:19])
        return (("youtube", date, user), 1)

    lines = lines.window(5, 5)
    lines = lines.map(lambda x: json.loads(x[1]))
    ytPost = lines.map(map)
    ytPost = ytPost.reduceByKey(lambda x, y: x + y)

    user = lines.map(userMap)
    user = user.transform(lambda rdd: rdd.distinct())
    user = user.map(lambda x: ((x[0][0], x[0][1]), x[1]))
    user = user.reduceByKey(lambda x, y: x + y)

    ytPost = ytPost.join(user).map(lambda x: (x[0], x[1][0], x[1][1]))
    ytPost = ytPost.map(lambda x: (x[0][0], x[0][1], x[1], x[2]))
    # ytPost = masukin ke postgre
    return ytPost

result = getYT(lines)
# Print
result.pprint()
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2023-04-17 23:03:45
-------------------------------------------
('youtube', '2023-04-04 17:05:00', 2, 2)

-------------------------------------------
Time: 2023-04-17 23:03:50
-------------------------------------------
('youtube', '2023-04-04 17:00:00', 1, 1)

-------------------------------------------
Time: 2023-04-17 23:03:55
-------------------------------------------

-------------------------------------------
Time: 2023-04-17 23:04:00
-------------------------------------------

-------------------------------------------
Time: 2023-04-17 23:04:05
-------------------------------------------

-------------------------------------------
Time: 2023-04-17 23:04:10
-------------------------------------------

-------------------------------------------
Time: 2023-04-17 23:04:15
-------------------------------------------

-------------------------------------------
Time: 2023-04-17 23:04:20
-------------------------------------------

KeyboardInterrupt: 